In [1]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
import os

In [2]:
spark = SparkSession.builder \
.appName("broadcast_vars") \
.master("local[*]") \
.config("spark.executor.memory", "2g") \
.config("spark.driver.memory", "4g") \
.getOrCreate()

23/04/16 00:42:54 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [3]:
sc = spark.sparkContext
sc.setLogLevel("ERROR")

In [7]:
current_dir = os.getcwd()
order_items_path = f"{current_dir}/order_items.csv"
products_path = f"{current_dir}/products.csv"

#order_items_rawRDD = sc.textFile(order_items_path)
#products_rawRDD = sc.textFile(products_path)

In [8]:
def read_products():
    products_text_wrapper = open(products_path, "r", encoding="utf-8")
    products = products_text_wrapper.readlines()
    product_id_name = {}
    for line in products:
        if "productCategoryId" not in line:
            product_id = int(line.split(",")[0])
            product_name = line.split(",")[2]  
            product_id_name.update({product_id:product_name})
    return product_id_name
        

In [9]:
products = read_products()

In [10]:
broadcast_products = sc.broadcast(products)

In [11]:
broadcast_products.value.get(114)

"Nike Men's Fly Shorts 2.0"

In [13]:
order_items_rdd = sc.textFile(order_items_path) \
.filter(lambda line: "orderItemOrderId" not in line)

order_items_rdd.take(5)

['1,1,957,1,299.98,299.98',
 '2,2,1073,1,199.99,199.99',
 '3,2,502,5,250.0,50.0',
 '4,2,403,1,129.99,129.99',
 '5,4,897,2,49.98,24.99']

In [14]:
def make_order_items_pair_rdd(line: str):
    orderItemProductId = int(line.split(",")[2])
    orderItemSubTotal = float(line.split(",")[4])
    return (orderItemProductId, orderItemSubTotal)

In [31]:
order_items_pairRDD = order_items_rdd.map(make_order_items_pair_rdd)
order_items_pairRDD.take(5)

[(957, 299.98), (1073, 199.99), (502, 250.0), (403, 129.99), (897, 49.98)]

In [38]:
sorted_ordersRDD = order_items_pairRDD \
.reduceByKey(lambda x,y: (x+y)) \
.map(lambda x: (x[1], x[0])) \
.sortByKey(ascending=False) \
.map(lambda x: (x[1], x[0]))

In [39]:
sorted_ordersRDD.take(10)

[(1004, 6929653.499999708),
 (365, 4421143.019999639),
 (957, 4118425.419999785),
 (191, 3667633.1999997487),
 (502, 3147800.0),
 (1073, 3099844.999999871),
 (403, 2891757.5399998166),
 (1014, 2888993.9399996493),
 (627, 1269082.649999932),
 (565, 67830.0)]

In [47]:
sorted_orders_with_product_name = sorted_ordersRDD \
.map(lambda x: (broadcast_products.value.get(x[0]), x[1]))

In [48]:
sorted_orders_with_product_name.take(10)

[('Field & Stream Sportsman 16 Gun Fire Safe', 6929653.499999708),
 ('Perfect Fitness Perfect Rip Deck', 4421143.019999639),
 ("Diamondback Women's Serene Classic Comfort Bi", 4118425.419999785),
 ("Nike Men's Free 5.0+ Running Shoe", 3667633.1999997487),
 ("Nike Men's Dri-FIT Victory Golf Polo", 3147800.0),
 ('Pelican Sunstream 100 Kayak', 3099844.999999871),
 ("Nike Men's CJ Elite 2 TD Football Cleat", 2891757.5399998166),
 ("O'Brien Men's Neoprene Life Vest", 2888993.9399996493),
 ("Under Armour Girls' Toddler Spine Surge Runni", 1269082.649999932),
 ('adidas Youth Germany Black/Red Away Match Soc', 67830.0)]